# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [17]:
import matplotlib.pyplot as plt
import os
import random
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

path = "Dataset"
print("Path to dataset files:", path)

Path to dataset files: Dataset


In [18]:
import os
import cv2
import numpy as np

# Đường dẫn tới thư mục Altered và Real
image_dir_altered = "Dataset/Altered"
image_dir_real = "Dataset/Real"

# Duyệt qua tất cả các ảnh trong cấu trúc thư mục phức tạp
def get_all_images(base_dir):
    all_images = []
    for root, dirs, files in os.walk(base_dir):  # Duyệt qua thư mục và tệp
        for file in files:
            if file.endswith('.BMP'):  # Chỉ lấy các tệp có đuôi .BMP
                full_path = os.path.join(root, file)
                all_images.append(full_path)  # Lưu đường dẫn đầy đủ
    return all_images

# Lấy danh sách tất cả các ảnh
all_real_images = get_all_images(image_dir_real)
all_altered_images = get_all_images(image_dir_altered)

# Kiểm tra số lượng ảnh
print(f"Found {len(all_real_images)} real images.")
print(f"Found {len(all_altered_images)} altered images.")

# Duyệt qua từng ảnh để xử lý
def process_and_plot(image_path, base_dir, output_base_dir):
    # Đọc ảnh
    image = cv2.imread(image_path)

    # Xử lý cơ bản (resize, denoise, vv.)
    resized_image = cv2.resize(image, (0, 0), fx=7, fy=7, interpolation=cv2.INTER_CUBIC)
    denoised_image = cv2.fastNlMeansDenoising(resized_image, None, 7, 7, 21)

    # Chuyển sang ảnh xám
    gray_image = cv2.cvtColor(denoised_image, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced_image = clahe.apply(gray_image)

    # Adaptive thresholding
    binary_image = cv2.adaptiveThreshold(enhanced_image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

    # Minutiae extraction
    minutiae_visualization = np.dstack((binary_image, binary_image, binary_image))
    minutiae_points = []
    rows, cols = binary_image.shape
    for y in range(1, rows - 1):
        for x in range(1, cols - 1):
            if binary_image[y, x] == 255:
                neighborhood = binary_image[y - 1:y + 2, x - 1:x + 2]
                ridge_count = np.sum(neighborhood == 255)
                if ridge_count == 2:
                    minutiae_points.append((x, y))
                    cv2.circle(minutiae_visualization, (x, y), 3, (0, 0, 255), -1)
                elif ridge_count == 4:
                    minutiae_points.append((x, y))
                    cv2.circle(minutiae_visualization, (x, y), 3, (0, 255, 0), -1)

    # Lưu kết quả minutiae visualization
    relative_path = os.path.relpath(image_path, base_dir)  # Tính đường dẫn tương đối từ thư mục gốc
    output_dir = os.path.join(output_base_dir, os.path.dirname(relative_path))  # Tạo cấu trúc thư mục kết quả

    # Đảm bảo thư mục đầu ra tồn tại
    os.makedirs(output_dir, exist_ok=True)

    # Lưu hình ảnh minutiae visualization
    image_name = os.path.splitext(os.path.basename(image_path))[0]  # Lấy tên ảnh không có phần mở rộng
    cv2.imwrite(os.path.join(output_dir, f"{image_name}_minutiae_visualization.jpg"), minutiae_visualization)

    # Lưu hình ảnh enhanced
    cv2.imwrite(os.path.join(output_dir, f"{image_name}_enhanced.jpg"), enhanced_image)

    # Tính orientation binary
    gradient_x = cv2.Sobel(binary_image, cv2.CV_64F, 1, 0, ksize=5)
    gradient_y = cv2.Sobel(binary_image, cv2.CV_64F, 0, 1, ksize=5)
    orientation_binary = cv2.phase(gradient_x, gradient_y, angleInDegrees=True)

    orientation_binary_normalized = cv2.normalize(orientation_binary, None, 0, 255, cv2.NORM_MINMAX).astype('uint8')

    # Lưu hình ảnh binary
    cv2.imwrite(os.path.join(output_dir, f"{image_name}_binary.jpg"), binary_image)

# Xử lý tất cả các ảnh trong Altered
for img_path in all_altered_images:
    process_and_plot(img_path, "Dataset/Altered", "Output/Altered")

# Xử lý tất cả các ảnh trong Real
for img_path in all_real_images:
    process_and_plot(img_path, "Dataset/Real", "Output/Real")


Found 10 real images.
Found 87 altered images.


In [16]:
# Phần huấn luyện mô hình SVM
def extract_features(image_paths):
    features = []
    for image_path in image_paths:
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if image is not None:
            features.append(image.flatten())  # Chuyển ảnh thành vector
    return np.array(features)

# Lấy đặc trưng từ Altered và Real
altered_features = extract_features(all_altered_images)
real_features = extract_features(all_real_images)

# Tạo nhãn cho Altered (1) và Real (0)
altered_labels = np.ones(len(altered_features))
real_labels = np.zeros(len(real_features))

# Gộp đặc trưng và nhãn
X = np.vstack((altered_features, real_features))
y = np.hstack((altered_labels, real_labels))

# Huấn luyện mô hình SVM
svm_model = SVC(kernel='linear', probability=True)
svm_model.fit(X, y)

# Lưu mô hình đã huấn luyện
import joblib
joblib.dump(svm_model, "fingerprint_svm_model.pkl")

# Kiểm tra mô hình
predictions = svm_model.predict(X)
accuracy = accuracy_score(y, predictions)
print(f"Model Accuracy: {accuracy * 100:.2f}%")


Model Accuracy: 100.00%
